# FBI background checks - States and categories

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import datetime as dt
import numpy as np

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

### Bless you @jsvine

In [5]:
url = "https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv"

### Read data

In [6]:
src = pd.read_csv(url)

In [7]:
src.iloc[0]

month                        2022-04
state                        Alabama
permit                       16150.0
permit_recheck                 236.0
handgun                      20593.0
long_gun                     12513.0
other                         1249.0
multiple                        1041
admin                            0.0
prepawn_handgun                 21.0
prepawn_long_gun                 2.0
prepawn_other                    0.0
redemption_handgun            2214.0
redemption_long_gun            852.0
redemption_other                 8.0
returned_handgun                37.0
returned_long_gun                0.0
returned_other                   0.0
rentals_handgun                  0.0
rentals_long_gun                 0.0
private_sale_handgun            16.0
private_sale_long_gun           19.0
private_sale_other               4.0
return_to_seller_handgun         0.0
return_to_seller_long_gun        1.0
return_to_seller_other           0.0
totals                         54956
N

In [8]:
src["year"] = src["month"].str[:4]

---

### AP states

In [9]:
ap_states = {
    "Alabama": "Ala.",
    "Alaska": "Alaska",
    "Arizona": "Ariz.",
    "Arkansas": "Ark.",
    "California": "Calif.",
    "Colorado": "Colo.",
    "Connecticut": "Conn.",
    "Delaware": "Del.",
    "Florida": "Fla.",
    "Georgia": "Ga.",
    "Hawaii": "Hawaii",
    "Idaho": "Iowa",
    "Illinois": "Idaho",
    "Indiana": "Ill.",
    "Iowa": "Ind.",
    "Kansas": "Kan.",
    "Kentucky": "Ky.",
    "Louisiana": "La.",
    "Maine": "Md.",
    "Maryland": "Mass.",
    "Massachusetts": "Maine",
    "Michigan": "Mich.",
    "Minnesota": "Minn.",
    "Mississippi": "Miss.",
    "Missouri": "Mo.",
    "Montana": "Mont.",
    "Nebraska": "Neb.",
    "Nevada": "Nev.",
    "New Hampshire": "N.H.",
    "New Jersey": "N.J.",
    "New Mexico": "N.M.",
    "New York": "N.Y.",
    "North Carolina": "N.C.",
    "North Dakota": "N.D.",
    "Ohio": "Ohio",
    "Oklahoma": "Okla.",
    "Oregon": "Ore.",
    "Pennsylvania": "Pa.",
    "Rhode Island": "R.I.",
    "South Carolina": "S.C.",
    "South Dakota": "S.D.",
    "Tennessee": "Tenn.",
    "Texas": "Texas",
    "Utah": "Utah",
    "Vermont": "Vt.",
    "Virginia": "Va.",
    "Washington": "Wash.",
    "West Virginia": "W.Va.",
    "Wisconsin": "Wis.",
    "Wyoming": "Wyo",
}

In [10]:
src["ap_state"] = src["state"].map(ap_states)

### Exclude permits figures

In [11]:
src["totals_no_permits"] = src[["handgun", "multiple", "long_gun"]].sum(axis=1)

In [12]:
src["year"] = src["month"].str[:4]

In [13]:
df = src[
    [
        "month",
        "year",
        "state",
        "ap_state",
        "handgun",
        "multiple",
        "long_gun",
        "totals_no_permits",
    ]
]

In [14]:
df.columns

Index(['month', 'year', 'state', 'ap_state', 'handgun', 'multiple', 'long_gun',
       'totals_no_permits'],
      dtype='object')

In [15]:
grouped = (
    df[df["year"] > "2008"]
    .groupby(["year"])
    .agg(
        {
            "handgun": sum,
            "long_gun": sum,
            "multiple": sum,
            "totals_no_permits": sum,
        }
    )
    .reset_index()
)
grouped

In [15]:
grouped.melt(index='year', var)

,year,handgun,long_gun,multiple,totals_no_permits
0,2009,3689604.0,4978641.0,235685,8903930.0
1,2010,3678001.0,4842184.0,180609,8700794.0
2,2011,4301389.0,5445966.0,219618,9966973.0
3,2012,5682963.0,6866225.0,230270,12779458.0
4,2013,6387502.0,7128798.0,241360,13757660.0
5,2014,6199243.0,5543371.0,225475,11968089.0
6,2015,7333808.0,5479441.0,242375,13055624.0
7,2016,8085498.0,5988511.0,257723,14331732.0
8,2017,7226979.0,5234757.0,236167,12697903.0
9,2018,6576111.0,4916533.0,257088,11749732.0


In [16]:
grouped[["year", "handgun", "long_gun", "multiple"]].to_csv(
    "data/processed/total_by_year.csv", index=False
)

In [17]:
exclude = [
    "Virgin Islands",
    "Guam",
    "Puerto Rico",
    "Mariana Islands",
    "District of Columbia",
]

In [18]:
grouped_states = (
    df[df["year"] > "2008"]
    .groupby(["year", "state", "ap_state"])
    .agg({"handgun": sum, "long_gun": sum, "totals_no_permits": sum})
    .reset_index()
)

In [19]:
change_states_2019_2020 = (
    grouped_states[
        (grouped_states["year"] > "2018") & (grouped_states["year"] < "2021")
    ]
    .pivot(index=["ap_state", "state"], columns="year", values="totals_no_permits")
    .reset_index()
)

In [20]:
change_states_2019_2020["change"] = round(
    (change_states_2019_2020["2020"] - change_states_2019_2020["2019"])
    / change_states_2019_2020["2019"]
    * 100,
    2,
)

In [21]:
state_change_export = change_states_2019_2020[
    ~change_states_2019_2020["state"].isin(exclude)
].sort_values("change", ascending=False)

In [22]:
state_change_export.to_csv("data/processed/state_change_2019-2020.csv", index=False)